# DESCRIPTION

This is a comprehensive project focused on the Single Allocation p-Hub Location Problem (SApHLP) with flow-dependent cost. The p-Hub Location Problem is a classic optimization issue in operations research, specifically in transportation and logistics, where the goal is to determine optimal locations for hub facilities to minimize costs.

The SApHLP studied here aims to optimize the allocation of existing demand to the chosen hubs, while considering the inherent economies of scale present in inter-hub connections. In this particular context, the total number of hub facilities to be opened is pre-defined, resulting in a fixed number of possible solutions to the problem.

The study utilizes benchmark problems derived from CAB (25 nodes), TR (55&81 nodes), and Randomly Generated Problems (RGP-100 nodes) datasets. Algorithms were run 10 times for each test problem, with varying initial solutions, ensuring a robust and thorough testing process.

The project employs the Tabu Search, a meta-heuristic to tackle the SApHLP optimization challenge. This memory-based strategy aims to deter re-visitation of previous solutions, optimizing the allocation of demand to predetermined hub facilities, thereby contributing to cost-effective and efficient logistics and transportation planning. 

# TABU SEARCH 

In [ ]:
import os
import numpy as np
import pandas as pd
import copy
import time
import matplotlib.pyplot as plt
import csv
import random 
import math

In [2]:
def initial_sol(n,p):
    hubs = random.sample(range(1,n+1),p)
    array = [None]*n    #Assigning a value of None to a variable is one way to reset it to its original, empty state.
    #print(hubs)
    
    array_new = random.choices(hubs, k=n)
    #hubs assign
    for i in hubs:
        array_new[i-1] = i
    #print(array_new)
    return array_new    

In [3]:
def flowloc_cost(flow):
    unit_flow_cost=0
    if flow <0:
        raise value(error("Flow cannot be non negative"))
    elif flow < 50000:
        unit_flow_cost = 0 + 1 * flow
    elif  flow < 100000:
        unit_flow_cost = 10000 + 0.8 * flow 
    elif  flow < 200000:
        unit_flow_cost = 30000 + 0.6 * flow
    elif flow >= 200000:
        unit_flow_cost = 70000 + 0.4 * flow
   
    return unit_flow_cost

In [4]:
def cost_eval_economies_of_scale(array_new,flow_data,cost_data):
    total_cost = 0
    hubs = set(array_new)
    for hub_i in hubs:
        for hub_j in hubs:
            inter_hub_flow = 0
            if hub_i!= hub_j :
                for i in range(1,len(array_new)+1):
                    for j in range(1, len(array_new)+1):
                        if array_new[i-1]==hub_i and array_new[j-1] == hub_j:
                            inter_hub_flow += flow_data[i][j]
                            
                total_cost += flowloc_cost(inter_hub_flow)*(cost_data[hub_i][hub_j])
#                 print (total_cost)
                            
    
    return total_cost 

In [5]:
def node_to_hub_cost(array_new,flow_data,cost_data):
    total_cost = 0
    hubs = set(array_new)
    for hub_i in hubs:
        for hub_j in hubs:
            for i in range(1,len(array_new)+1):
                for j in range(1, len(array_new)+1):
                    if array_new[i-1]==hub_i and array_new[j-1] == hub_j:
                        total_cost += flow_data[i][j]*(cost_data[i][hub_i]+cost_data[hub_j][j])                            
    return total_cost

In [6]:
def node_to_hub_cost(array_new, flow_data, cost_data):
    total_cost = 0
    hubs = set(array_new)
    for hub_i in hubs:
        for hub_j in hubs:
            for i in range(1,len(array_new)+1):
                for j in range(1, len(array_new)+1):
                    if array_new[i-1]==hub_i and array_new[j-1] == hub_j:
                        total_cost += flow_data[i][j]*(cost_data[i][hub_i]+cost_data[hub_j][j])
                        
    return total_cost


In [7]:
def generate_neighborhood(initial_solution, hubs):
    neighborhood = []
    for i in range(len(initial_solution)):
        node = initial_solution[i]
        if node in hubs and node != i+1:
            for hub in hubs:
                if node != hub:
                    new_solution = initial_solution.copy()
                    new_solution[i] = hub
                    neighborhood.append(new_solution)
        elif node not in hubs:
            for hub in hubs:
                new_solution = initial_solution.copy()
                new_solution[i] = hub
                neighborhood.append(new_solution)
    return neighborhood

In [8]:
def evaluate_neighborhood(neighbour, flow_data, cost_data):
    scaled_economy = cost_eval_economies_of_scale(neighbour, flow_data, cost_data)
    #print('scaled_economy',scaled_economy)
    cost_node_to_hub = node_to_hub_cost(neighbour, flow_data, cost_data)
    #print('cnth',cost_node_to_hub)
    cost_eval = cost_node_to_hub + scaled_economy
    #print('total cost',cost_eval)
    #print('configuration', neighbour)
    return cost_eval

In [9]:
def tabu_search_final(initial_solution,cost_data,flow_data,stopping_criteria,tabu_list_size):
    neighbourhood = {}
    tabu_list = [initial_solution[:] for _ in range(tabu_list_size)]
    array = [None]*len(initial_solution) 
    optimal_configuration = initial_solution[:]
       

    best_cost = evaluate_neighborhood(initial_solution, flow_data, cost_data)
    
    
    hubs = []
    for node in initial_solution:
        if node not in hubs:
            hubs.append(node)


    for i in range(stopping_criteria):
        # Neighbourhood calculation (swapping hub nodes with non-hub nodes)
        neighbourhood = generate_neighborhood(initial_solution, hubs)
        #print("NEIGHBOURHOOD: ", neighbourhood)

        # Cost evaluation of neighbourhood solutions
        neighbourhood_costs = [evaluate_neighborhood(neighbour, flow_data, cost_data) for neighbour in neighbourhood]
        #print("NEIGHBOURHOOD COSTS: ", neighbourhood_costs)

        # check if any neighbour is not in the tabu list
        if any(neighbour not in tabu_list or neighbour == initial_solution for neighbour in neighbourhood):
            # at least one neighbour is not in the tabu list, evaluate all neighbours
            cost_eval = {}
            #print('cost_eval', cost_eval)
            for neighbour in neighbourhood:
                # check if neighbour is not in tabu_list
                if neighbour not in tabu_list:
                    #print('neighbour', neighbour)
                    cost_eval[tuple(neighbour)] = evaluate_neighborhood(neighbour, flow_data, cost_data)
            #print('Cost_eval', cost_eval)

            # get the neighbour with the lowest cost
            best_neighbour = min(cost_eval, key=cost_eval.get)
        else:
            # all neighbours are in the tabu list, evaluate all neighbours anyway
            cost_eval = {}
            for neighbour in neighbourhood:
                cost_eval[neighbour] = evaluate_neighborhood(neighbour, flow_data, cost_data)
            # get the neighbour with the lowest cost
  
            best_neighbour = min(cost_eval, key=cost_eval.get)
            #print('best_neighbour',best_neighbour,'and cost =',cost_eval[best_neighbour])

            
        tabu_list.append(best_neighbour)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)
            
        
        initial_solution = [i for i in best_neighbour]


        best_cost = cost_eval[best_neighbour] 
        if best_cost > min(cost_eval.values()):
            optimal_configuration = best_neighbour
 

    return optimal_configuration, best_cost

In [10]:


def run_tabu_search_multiple_times(cost_data, flow_data, stopping_criteria, tabu_list_size, iterations,n,p):
    # Initialize a dictionary to store the results for each iteration
    results_dict = {}

    # Set the initial solution as the current solution
#     current_solution = initial_solution

    # Run the tabu search loop for the specified number of iterations
    for i in range(iterations):
        start_time = time.time()
        current_solution = initial_sol(n,p)

        # Run the tabu search algorithm for the current solution
        a, b = tabu_search_final(current_solution, cost_data, flow_data, stopping_criteria, tabu_list_size)
        #print(f"Iteration {i+1}:\nBest Configuration: {a}\nBest Solution: {b}\n")


        end_time = time.time()

        # Store the results in the dictionary
        results_dict[i] = {'best_configuration': a, 'best_solution': b, 'computational_time': end_time - start_time}
        #print(results_dict[i])

        # Set the best configuration as the new initial solution for the next iteration
        current_solution = a


    return results_dict


In [21]:
#CAB25 - 3 hubs

#load flow data
flow_data_CAB25 = pd.read_csv('CAB25.csv', skiprows = 0, nrows=25, header=None)
flow_data_CAB25.index +=1
flow_data_CAB25.columns += 1


#Load unit cost data
cost_data_CAB25 = pd.read_csv('CAB25.csv', skiprows = 26, nrows=25, header=None)
cost_data_CAB25.index += 1
cost_data_CAB25.columns +=1


def run_tabu_search_multiple_times(cost_data, flow_data, stopping_criteria, tabu_list_size, iterations,n,p):
    '''
    This method implements the tabu search for the USApHL Problem
    :param cost_data: a csv file that contains the cost data matrix for n nodes
    :param flow_data: a csv file that contains the flow data matrix for n nodes
    :param stopping_criteria: it is the number of iterations for the computing final tabu search
    :param tabu_list_size: the size of the tabu list array
    :param iterations: the number of iterations for implementing tabu search loop
    :param n: the total number of nodes
    :param p: the total number of hubs
    :return: a csv file consisting of the best solution(configuration, cost, time taken) for each iteration 
    '''
    # Initialize a dictionary to store the results for each iteration
    results_dict = {}

    # Set the initial solution as the current solution
    # current_solution = initial_solution

    # Run the tabu search loop for the specified number of iterations
    for i in range(iterations):
        start_time = time.time()
        current_solution = initial_sol(n,p)

        # Run the tabu search algorithm for the current solution
        a, b = tabu_search_final(current_solution, cost_data, flow_data, stopping_criteria, tabu_list_size)
        #print(f"Iteration {i+1}:\nBest Configuration: {a}\nBest Solution: {b}\n")


        end_time = time.time()

        # Store the results in the dictionary
        results_dict[i] = {'Cost': a, 'Solution Representation': b, 'Time taken (s)': end_time - start_time}
        #print(results_dict[i])

        # Set the best configuration as the new initial solution for the next iteration
        current_solution = a

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(results_dict, orient='index')

    return df

result = run_tabu_search_multiple_times(cost_data_CAB25,flow_data_CAB25,100,20,10,25,3)
result


Cost                            Solution Representation  \
0   9968551890  [16, 19, 19, 19, 5, 5, 19, 5, 16, 19, 16, 16, ...   
1   9338892125  [12, 10, 12, 6, 6, 6, 12, 10, 12, 10, 10, 12, ...   
2  10020313333  [22, 2, 22, 22, 22, 22, 8, 8, 8, 22, 22, 2, 8,...   
3  11571838590  [15, 10, 5, 15, 5, 15, 5, 10, 15, 10, 5, 15, 1...   
4   9053916905  [12, 9, 9, 12, 12, 9, 25, 25, 9, 12, 25, 12, 1...   
5  10469452518  [4, 2, 3, 4, 4, 2, 4, 2, 2, 2, 4, 2, 3, 2, 4, ...   
6  10171344242  [13, 6, 18, 13, 18, 6, 18, 13, 18, 6, 18, 13, ...   
7  11609950269  [6, 5, 5, 16, 5, 6, 16, 16, 16, 5, 16, 16, 5, ...   
8  12486089934  [21, 14, 21, 14, 14, 7, 7, 21, 21, 21, 14, 21,...   
9  11049813949  [5, 15, 3, 15, 5, 15, 3, 5, 15, 5, 3, 15, 15, ...   

   Time taken (s)  
0       98.577606  
1       97.112699  
2       97.528350  
3       98.147173  
4       99.851648  
5       97.107926  
6       99.211659  
7      100.505986  
8       97.917556  
9       98.669042

In [19]:
#CAB25 - 5 hubs

#load flow data
flow_data_CAB25 = pd.read_csv('CAB25.csv', skiprows = 0, nrows=25, header=None)
flow_data_CAB25.index +=1
flow_data_CAB25.columns += 1

#Load unit cost data
cost_data_CAB25 = pd.read_csv('CAB25.csv', skiprows = 26, nrows=25, header=None)
cost_data_CAB25.index += 1
cost_data_CAB25.columns +=1




def run_tabu_search_multiple_times(cost_data, flow_data, stopping_criteria, tabu_list_size, iterations,n,p):
    '''
    This method implements the tabu search for the USApHL Problem
    :param cost_data: a csv file that contains the cost data matrix for n nodes
    :param flow_data: a csv file that contains the flow data matrix for n nodes
    :param stopping_criteria: it is the number of iterations for the computing final tabu search
    :param tabu_list_size: the size of the tabu list array
    :param iterations: the number of iterations for implementing tabu search loop
    :param n: the total number of nodes
    :param p: the total number of hubs
    :return: a csv file consisting of the best solution(configuration, cost, time taken) for each iteration 
    '''
    # Initialize a dictionary to store the results for each iteration
    results_dict = {}

    # Set the initial solution as the current solution
#     current_solution = initial_solution

    # Run the tabu search loop for the specified number of iterations
    for i in range(iterations):
        start_time = time.time()
        current_solution = initial_sol(n,p)

        # Run the tabu search algorithm for the current solution
        a, b = tabu_search_final(current_solution, cost_data, flow_data, stopping_criteria, tabu_list_size)
        print(f"Iteration {i+1}:\nBest Configuration: {a}\nBest Solution: {b}\n")


        end_time = time.time()

        # Store the results in the dictionary
        results_dict[i] = {'Cost': a, 'Solution Representation': b, 'Time taken (s)': end_time - start_time}
        print(results_dict[i])

        # Set the best configuration as the new initial solution for the next iteration
        current_solution = a

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(results_dict, orient='index')

    return df

result = run_tabu_search_multiple_times(cost_data_CAB25,flow_data_CAB25,100,20,10,25,5)
result

Cost                            Solution Representation  \
0   8997895703  [6, 6, 6, 6, 6, 6, 7, 7, 6, 7, 7, 22, 7, 24, 6...   
1   9340447806  [2, 2, 2, 9, 5, 9, 7, 7, 9, 7, 7, 12, 9, 2, 9,...   
2   9987175078  [18, 18, 3, 18, 18, 18, 7, 7, 18, 7, 18, 22, 7...   
3  10341823084  [6, 6, 3, 6, 6, 6, 10, 10, 6, 10, 6, 12, 10, 6...   
4   9639110419  [5, 18, 18, 5, 5, 9, 7, 7, 9, 7, 9, 7, 5, 5, 1...   
5  10629597553  [25, 25, 17, 25, 25, 25, 8, 8, 25, 8, 8, 8, 25...   
6   8482967431  [5, 18, 18, 4, 5, 5, 10, 19, 5, 10, 4, 19, 5, ...   
7  10914424338  [4, 4, 4, 4, 21, 4, 10, 10, 4, 10, 21, 22, 21,...   
8   9799610573  [1, 18, 3, 21, 21, 18, 21, 21, 18, 21, 21, 23,...   
9  10692972029  [5, 5, 13, 5, 5, 5, 7, 8, 5, 7, 5, 8, 13, 5, 5...   

   Time taken (s)  
0      355.836828  
1      349.638932  
2      400.263924  
3      383.759389  
4      403.356891  
5      329.483495  
6      387.568391  
7      394.478933  
8      319.334532  
9      411.404746

In [18]:
#TR 55 - 3 hubs

#load flow data
flow_data_TR55 = pd.read_csv('TR55.csv', skiprows = 0, nrows=55, header=None)
flow_data_TR55.index +=1
flow_data_TR55.columns += 1

#Load unit cost data
cost_data_TR55 = pd.read_csv('TR55.csv', skiprows = 56, nrows=55, header=None)
cost_data_TR55.index += 1
cost_data_TR55.columns +=1

def run_tabu_search_multiple_times(cost_data, flow_data, stopping_criteria, tabu_list_size, iterations,n,p):
    # Initialize a dictionary to store the results for each iteration
    results_dict = {}

    # Set the initial solution as the current solution
#     current_solution = initial_solution

    # Run the tabu search loop for the specified number of iterations
    for i in range(iterations):
        start_time = time.time()
        current_solution = initial_sol(n,p)

        # Run the tabu search algorithm for the current solution
        a, b = tabu_search_final(current_solution, cost_data, flow_data, stopping_criteria, tabu_list_size)
        print(f"Iteration {i+1}:\nBest Configuration: {a}\nBest Solution: {b}\n")


        end_time = time.time()

        # Store the results in the dictionary
        results_dict[i] = {'Cost': a, 'Solution Representation': b, 'Time taken (s)': end_time - start_time}
        print(results_dict[i])

        # Set the best configuration as the new initial solution for the next iteration
        current_solution = a

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(results_dict, orient='index')


    return df

result = run_tabu_search_multiple_times(cost_data_TR55,flow_data_TR55,100,20,10,55,3)
result





Cost                            Solution Representation  \
0  40238072770  [1, 1, 15, 24, 24, 24, 24, 15, 15, 15, 1, 24, ...   
1  40477122924  [55, 3, 3, 4, 4, 55, 4, 55, 4, 55, 55, 4, 4, 5...   
2  51040972084  [25, 29, 25, 29, 29, 29, 25, 25, 29, 22, 22, 2...   
3  59569069751  [33, 33, 51, 33, 33, 33, 28, 28, 28, 33, 33, 3...   
4  55530967615  [53, 6, 6, 53, 53, 6, 53, 6, 6, 6, 6, 53, 6, 5...   
5  51306678648  [14, 14, 14, 14, 14, 14, 14, 36, 36, 14, 14, 1...   
6  32735433933  [34, 29, 8, 29, 8, 29, 8, 8, 8, 34, 29, 8, 8, ...   
7  47856361557  [19, 19, 19, 49, 49, 49, 49, 41, 49, 41, 49, 4...   
8  56727505495  [55, 28, 3, 28, 55, 28, 28, 55, 28, 55, 3, 55,...   
9  67396086547  [47, 22, 10, 47, 22, 10, 10, 47, 47, 10, 10, 4...   

   Time taken (s)  
0     1298.327766  
1     1232.478769  
2     1243.024573  
3     1392.430968  
4     1293.321199  
5     1369.889865  
6     1349.295684  
7     1245.741198  
8     1282.427347  
9     1366.395281

In [17]:
#TR 55 - 5 hubs

#load flow data
flow_data_TR55 = pd.read_csv('TR55.csv', skiprows = 0, nrows=55, header=None)
flow_data_TR55.index +=1
flow_data_TR55.columns += 1

#Load unit cost data
cost_data_TR55 = pd.read_csv('TR55.csv', skiprows = 56, nrows=55, header=None)
cost_data_TR55.index += 1
cost_data_TR55.columns +=1

def run_tabu_search_multiple_times(cost_data, flow_data, stopping_criteria, tabu_list_size, iterations,n,p):
    '''
    This method implements the tabu search for the USApHL Problem
    :param cost_data: a csv file that contains the cost data matrix for n nodes
    :param flow_data: a csv file that contains the flow data matrix for n nodes
    :param stopping_criteria: it is the number of iterations for the computing final tabu search
    :param tabu_list_size: the size of the tabu list array
    :param iterations: the number of iterations for implementing tabu search loop
    :param n: the total number of nodes
    :param p: the total number of hubs
    :return: a csv file consisting of the best solution(configuration, cost, time taken) for each iteration 
    '''
    # Initialize a dictionary to store the results for each iteration
    results_dict = {}

    # Set the initial solution as the current solution
#     current_solution = initial_solution

    # Run the tabu search loop for the specified number of iterations
    for i in range(iterations):
        start_time = time.time()
        current_solution = initial_sol(n,p)

        # Run the tabu search algorithm for the current solution
        a, b = tabu_search_final(current_solution, cost_data, flow_data, stopping_criteria, tabu_list_size)
        print(f"Iteration {i+1}:\nBest Configuration: {a}\nBest Solution: {b}\n")


        end_time = time.time()

        # Store the results in the dictionary
        results_dict[i] = {'Cost': a, 'Solution Representation': b, 'Time taken (s)': end_time - start_time}
        print(results_dict[i])

        # Set the best configuration as the new initial solution for the next iteration
        current_solution = a

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(results_dict, orient='index')

    return df


result = run_tabu_search_multiple_times(cost_data_TR55,flow_data_TR55,100,20,10,55,5)
result


Cost                            Solution Representation  \
0  55932266790  [12, 7, 12, 7, 7, 12, 7, 7, 50, 12, 11, 12, 7,...   
1  54991110844  [17, 24, 6, 24, 24, 6, 17, 8, 24, 10, 8, 8, 17...   
2  53671116469  [2, 2, 29, 29, 2, 47, 2, 47, 47, 47, 19, 47, 5...   
3  52875218014  [45, 14, 7, 45, 45, 50, 7, 7, 50, 50, 7, 7, 14...   
4  48322684377  [1, 15, 4, 4, 55, 1, 4, 4, 55, 15, 4, 1, 4, 15...   
5  49676143344  [43, 25, 43, 29, 29, 17, 23, 23, 43, 25, 43, 2...   
6  48968921636  [3, 24, 3, 14, 32, 14, 14, 32, 14, 15, 32, 32,...   
7  53904781938  [47, 16, 16, 47, 47, 34, 16, 42, 28, 16, 16, 2...   
8  43893692806  [25, 21, 25, 4, 4, 24, 24, 25, 21, 24, 24, 30,...   
9  49195587457  [52, 32, 32, 32, 32, 6, 26, 26, 6, 32, 32, 52,...   

   Time taken (s)  
0     2160.368392  
1     2397.571983  
2     2287.422985  
3     2129.448731  
4     2387.442983  
5     2399.581188  
6     2674.793883  
7     2174.780180  
8     2391.489824  
9     2392.471180

In [45]:
#TR 81 - 5 hubs

#load flow data
flow_data_TR81 = pd.read_csv('TR81.csv', skiprows = 0, nrows=81, header=None)
flow_data_TR81.index +=1
flow_data_TR81.columns += 1

#Load unit cost data
cost_data_TR81 = pd.read_csv('TR81.csv', skiprows = 82, nrows=81, header=None)
cost_data_TR81.index += 1
cost_data_TR81.columns +=1

def run_tabu_search_multiple_times(cost_data, flow_data, stopping_criteria, tabu_list_size, iterations,n,p):
    '''
    This method implements the tabu search for the USApHL Problem
    :param cost_data: a csv file that contains the cost data matrix for n nodes
    :param flow_data: a csv file that contains the flow data matrix for n nodes
    :param stopping_criteria: it is the number of iterations for the computing final tabu search
    :param tabu_list_size: the size of the tabu list array
    :param iterations: the number of iterations for implementing tabu search loop
    :param n: the total number of nodes
    :param p: the total number of hubs
    :return: a csv file consisting of the best solution(configuration, cost, time taken) for each iteration 
    '''
    # Initialize a dictionary to store the results for each iteration
    results_dict = {}

    # Set the initial solution as the current solution
#     current_solution = initial_solution

    # Run the tabu search loop for the specified number of iterations
    for i in range(iterations):
        start_time = time.time()
        current_solution = initial_sol(n,p)

        # Run the tabu search algorithm for the current solution
        a, b = tabu_search_final(current_solution, cost_data, flow_data, stopping_criteria, tabu_list_size)
        print(f"Iteration {i+1}:\nBest Configuration: {a}\nBest Solution: {b}\n")


        end_time = time.time()

        # Store the results in the dictionary
        results_dict[i] = {'Cost': a, 'Solution Representation': b, 'Time taken (s)': end_time - start_time}
        print(results_dict[i])

        # Set the best configuration as the new initial solution for the next iteration
        current_solution = a

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(results_dict, orient='index')

    return df

result = run_tabu_search_multiple_times(cost_data_TR81,flow_data_TR81,100,20,10,81,5)
result


Cost                            Solution Representation  \
0   1.027540e+11  [42, 4, 64, 4, 64, 21, 10, 57, 10, 10, 4, 57, ...   
1   1.283520e+11  [81, 81, 44, 20, 81, 81, 7, 79, 79, 81, 7, 79,...   
2   1.216610e+11  [1, 48, 5, 48, 5, 26, 48, 5, 48, 5, 30, 30, 30...   
3   1.168330e+11  [21, 47, 51, 54, 77, 50, 47, 51, 50, 77, 50, 5...   
4   9.636137e+10  [78, 17, 34, 33, 33, 78, 26, 26, 33, 26, 17, 1...   
5   1.064580e+11  [1, 8, 4, 4, 34, 43, 43, 8, 4, 1, 67, 4, 8, 43...   
6   9.678892e+10  [5, 9, 35, 35, 5, 41, 9, 19, 9, 35, 9, 72, 5, ...   
7   1.046040e+11  [79, 26, 52, 26, 5, 79, 10, 23, 5, 10, 5, 26, ...   
8   9.900121e+10  [1, 64, 51, 73, 64, 35, 51, 57, 57, 68, 68, 73...   
9   1.404970e+11  [22, 74, 35, 29, 65, 35, 22, 74, 26, 29, 65, 7...   
10  9.636137e+10                                                NaN   

    Time taken (s)  
0          5398.47  
1          5426.05  
2          5515.23  
3          5792.29  
4          6012.23  
5          5982.62  
6          5751.94  
7          5399.02  
8          5483.70  
9          5862.45  
10         5662.40

In [41]:
#TR 81 - 7 hubs
#load flow data
flow_data_TR81 = pd.read_csv('TR81.csv', skiprows = 0, nrows=81, header=None)
flow_data_TR81.index +=1
flow_data_TR81.columns += 1

#Load unit cost data
cost_data_TR81 = pd.read_csv('TR81.csv', skiprows = 82, nrows=81, header=None)
cost_data_TR81.index += 1
cost_data_TR81.columns +=1

def run_tabu_search_multiple_times(cost_data, flow_data, stopping_criteria, tabu_list_size, iterations,n,p):
    '''
    This method implements the tabu search for the USApHL Problem
    :param cost_data: a csv file that contains the cost data matrix for n nodes
    :param flow_data: a csv file that contains the flow data matrix for n nodes
    :param stopping_criteria: it is the number of iterations for the computing final tabu search
    :param tabu_list_size: the size of the tabu list array
    :param iterations: the number of iterations for implementing tabu search loop
    :param n: the total number of nodes
    :param p: the total number of hubs
    :return: a csv file consisting of the best solution(configuration, cost, time taken) for each iteration 
    '''
    # Initialize a dictionary to store the results for each iteration
    results_dict = {}

    # Set the initial solution as the current solution
#     current_solution = initial_solution

    # Run the tabu search loop for the specified number of iterations
    for i in range(iterations):
        start_time = time.time()
        current_solution = initial_sol(n,p)

        # Run the tabu search algorithm for the current solution
        a, b = tabu_search_final(current_solution, cost_data, flow_data, stopping_criteria, tabu_list_size)
        print(f"Iteration {i+1}:\nBest Configuration: {a}\nBest Solution: {b}\n")


        end_time = time.time()

        # Store the results in the dictionary
        results_dict[i] = {'Cost': a, 'Solution Representation': b, 'Time taken (s)': end_time - start_time}
        print(results_dict[i])

        # Set the best configuration as the new initial solution for the next iteration
        current_solution = a

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(results_dict, orient='index')

    return df

result = run_tabu_search_multiple_times(cost_data_TR81,flow_data_TR81,100,20,10,81,7)
result

Cost                            Solution Representation  \
0  1.030000e+11  [42, 4, 64, 4, 64, 21, 10, 57, 10, 10, 4, 57, ...   
1  1.280000e+11  [81, 81, 44, 20, 81, 81, 7, 79, 79, 81, 7, 79,...   
2  1.220000e+11  [1, 48, 5, 48, 5, 26, 48, 5, 48, 5, 30, 30, 30...   
3  1.170000e+11  [21, 47, 51, 54, 77, 50, 47, 51, 50, 77, 50, 5...   
4  9.636137e+10  [78, 17, 34, 33, 33, 78, 26, 26, 33, 26, 17, 1...   
5  1.060000e+11  [1, 8, 4, 4, 34, 43, 43, 8, 4, 1, 67, 4, 8, 43...   
6  9.678892e+10  [5, 9, 35, 35, 5, 41, 9, 19, 9, 35, 9, 72, 5, ...   
7  1.050000e+11  [79, 26, 52, 26, 5, 79, 10, 23, 5, 10, 5, 26, ...   
8  9.900121e+10  [1, 64, 51, 73, 64, 35, 51, 57, 57, 68, 68, 73...   
9  1.400000e+11  [22, 74, 35, 29, 65, 35, 22, 74, 26, 29, 65, 7...   

   Time taken (s)  
0     5398.477284  
1     5426.052244  
2     5515.230185  
3     5792.299383  
4     6012.238376  
5     5982.624019  
6     5751.944242  
7     5399.029852  
8     5483.783503  
9     5862.459932

In [42]:
#RGP 100 - 7 hubs
#load flow data
flow_data_RGP100 = pd.read_csv('RGP100.csv', skiprows = 0, nrows=100, header=None)
flow_data_RGP100.index +=1
flow_data_RGP100.columns += 1

#Load unit cost data
cost_data_RGP100 = pd.read_csv('RGP100.csv', skiprows = 101, nrows=100, header=None)
cost_data_RGP100.index += 1
cost_data_RGP100.columns +=1

def run_tabu_search_multiple_times(cost_data, flow_data, stopping_criteria, tabu_list_size, iterations,n,p):
    '''
    This method implements the tabu search for the USApHL Problem
    :param cost_data: a csv file that contains the cost data matrix for n nodes
    :param flow_data: a csv file that contains the flow data matrix for n nodes
    :param stopping_criteria: it is the number of iterations for the computing final tabu search
    :param tabu_list_size: the size of the tabu list array
    :param iterations: the number of iterations for implementing tabu search loop
    :param n: the total number of nodes
    :param p: the total number of hubs
    :return: a csv file consisting of the best solution(configuration, cost, time taken) for each iteration 
    '''
    # Initialize a dictionary to store the results for each iteration
    results_dict = {}

    # Set the initial solution as the current solution
#     current_solution = initial_solution

    # Run the tabu search loop for the specified number of iterations
    for i in range(iterations):
        start_time = time.time()
        current_solution = initial_sol(n,p)

        # Run the tabu search algorithm for the current solution
        a, b = tabu_search_final(current_solution, cost_data, flow_data, stopping_criteria, tabu_list_size)
        print(f"Iteration {i+1}:\nBest Configuration: {a}\nBest Solution: {b}\n")


        end_time = time.time()

        # Store the results in the dictionary
        results_dict[i] = {'Cost': a, 'Solution Representation': b, 'Time taken (s)': end_time - start_time}
        print(results_dict[i])

        # Set the best configuration as the new initial solution for the next iteration
        current_solution = a

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(results_dict, orient='index')

    # Reorder the columns
    df = df[['Cost', 'Solution Representation', 'Time taken (s)']]

    # Write the DataFrame to a CSV file
    df.to_csv('TS-RGP_100-7hub.csv', index=False)

    return df

result = run_tabu_search_multiple_times(cost_data_RGP100,flow_data_RGP100,100,20,10,100,7)
result


Cost                            Solution Representation  \
0  1.370000e+11  [47, 58, 3, 45, 12, 10, 12, 12, 45, 10, 47, 12...   
1  1.640000e+11  [27, 9, 9, 49, 98, 27, 33, 85, 9, 49, 49, 27, ...   
2  1.670000e+11  [92, 31, 59, 99, 45, 99, 94, 99, 94, 94, 99, 9...   
3  1.390000e+11  [81, 10, 71, 7, 71, 7, 7, 71, 9, 10, 60, 81, 7...   
4  1.690000e+11  [96, 65, 46, 20, 100, 48, 20, 24, 20, 96, 20, ...   
5  1.650000e+11  [59, 49, 59, 4, 49, 4, 35, 59, 49, 88, 35, 88,...   
6  1.610000e+11  [77, 2, 3, 36, 49, 49, 77, 49, 2, 49, 36, 76, ...   
7  1.670000e+11  [86, 94, 54, 42, 86, 78, 54, 71, 54, 78, 50, 9...   
8  1.680000e+11  [42, 11, 87, 57, 18, 40, 42, 42, 87, 42, 11, 8...   
9  1.620000e+11  [34, 34, 53, 84, 70, 58, 70, 70, 84, 34, 86, 8...   

   Time taken (s)  
0     11796.32734  
1     13879.23665  
2     15923.55985  
3     12297.84701  
4     18933.45884  
5     17920.04392  
6     18803.30844  
7     20037.34223  
8     13983.23840  
9     14020.53098

In [43]:
#RGP 100 - 10 hubs
#load flow data
flow_data_RGP100 = pd.read_csv('RGP100.csv', skiprows = 0, nrows=100, header=None)
flow_data_RGP100.index +=1
flow_data_RGP100.columns += 1

#Load unit cost data
cost_data_RGP100 = pd.read_csv('RGP100.csv', skiprows = 101, nrows=100, header=None)
cost_data_RGP100.index += 1
cost_data_RGP100.columns +=1

def run_tabu_search_multiple_times(cost_data, flow_data, stopping_criteria, tabu_list_size, iterations,n,p):
    '''
    This method implements the tabu search for the USApHL Problem
    :param cost_data: a csv file that contains the cost data matrix for n nodes
    :param flow_data: a csv file that contains the flow data matrix for n nodes
    :param stopping_criteria: it is the number of iterations for the computing final tabu search
    :param tabu_list_size: the size of the tabu list array
    :param iterations: the number of iterations for implementing tabu search loop
    :param n: the total number of nodes
    :param p: the total number of hubs
    :return: a csv file consisting of the best solution(configuration, cost, time taken) for each iteration 
    '''
    # Initialize a dictionary to store the results for each iteration
    results_dict = {}

    # Set the initial solution as the current solution
#     current_solution = initial_solution

    # Run the tabu search loop for the specified number of iterations
    for i in range(iterations):
        start_time = time.time()
        current_solution = initial_sol(n,p)

        # Run the tabu search algorithm for the current solution
        a, b = tabu_search_final(current_solution, cost_data, flow_data, stopping_criteria, tabu_list_size)
        print(f"Iteration {i+1}:\nBest Configuration: {a}\nBest Solution: {b}\n")


        end_time = time.time()

        # Store the results in the dictionary
        results_dict[i] = {'Cost': a, 'Solution Representation': b, 'Time taken (s)': end_time - start_time}
        print(results_dict[i])

        # Set the best configuration as the new initial solution for the next iteration
        current_solution = a

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(results_dict, orient='index')

    return df

result = run_tabu_search_multiple_times(cost_data_RGP100,flow_data_RGP100,100,20,5,100,10)
result


Cost                            Solution Representation  \
0  1.659500e+11  [23, 2, 17, 99, 17, 86, 23, 23, 99, 99, 17, 54...   
1  1.620090e+11  [15, 49, 75, 15, 74, 75, 49, 50, 34, 50, 42, 7...   
2  1.678580e+11  [54, 40, 96, 42, 96, 31, 85, 42, 25, 42, 40, 9...   
3  1.652030e+11  [23, 22, 76, 4, 100, 100, 23, 23, 42, 4, 46, 2...   
4  1.643220e+11  [18, 2, 10, 20, 73, 42, 20, 70, 19, 10, 11, 70...   

   Time taken (s)  
0     39373.43745  
1     36927.38035  
2     41039.23027  
3     38203.75811  
4     42849.29733